# Working with Window Functions in PySPark

Window Functions helps us to compare current row with other rows in the same dataframe, calculating running totals, sequencing of events and sessionization of transactions etc. This notebook is compiled from the following blog posts and my additional insights and examples from my own understanding and day to day work:

+ https://databricks.com/blog/2015/07/15/introducing-window-functions-in-spark-sql.html 
+ http://alvinhenrick.com/2017/05/16/apache-spark-analytical-window-functions/ 
+ https://jaceklaskowski.gitbooks.io/mastering-spark-sql/spark-sql-functions-windows.html 
+ https://johnpaton.net/posts/forward-fill-spark/

## Table of contents



| dd | dd |
|----|----|
|  [1.1 Rank salary within each department](#1.1-Rank-salary-within-each-department:)  |  WINDOW, RANK  |
|    |    |
|    |    |

A brief synopsis of what each use case is and what functionality of the SPARK SQL or DF API does it touch on.

<table align="left">
   <tr>
      <th style="text-align: left;">Section</th>
      <th style="text-align: left;">Demonstrates</th>
   </tr>
   <tr>
      <td style="text-align: left;">[1.1 Rank salary within each department](#1.1-Rank-salary-within-each-department:)</td>
      <td style="text-align: left;">WINDOW, RANK</td>
   </tr>
   <tr>
      <td style="text-align: left;">[1.2 Dense Rank salary within each department](#1.2-Dense-Rank-salary-within-each-department:)</td>
      <td style="text-align: left;">WINDOW, DENSE_RANK</td>
   </tr>
   <tr>
      <td style="text-align: left;">[1.3 Row Number within each department](#1.3-Row-Number-within-each-department:)</td>
      <td style="text-align: left;">WINDOW, ROW_NUMBER</td>
   </tr>
   <tr>
      <td style="text-align: left;">[1.4 Running Total Salary within each department](#1.4-Running-Total-Salary-within-each-department:)</td>
      <td style="text-align: left;">SUM, WINDOW</td>
   </tr>
   <tr>
      <td style="text-align: left;">[1.5 Next Salary within each department](#1.5-Next-Salary-within-each-department:)</td>
      <td style="text-align: left;">LEAD, WINDOW</td>
   </tr>
   <tr>
      <td style="text-align: left;">[1.6 Previous Salary within each department](#1.6-Previous-Salary-within-each-department:)</td>
      <td style="text-align: left;">LAG, WINDOW</td>
   </tr>
   <tr>
      <td style="text-align: left;">[1.7 First Salary within each department](#1.7-First-Salary-within-each-department:)</td>
      <td style="text-align: left;">FIRST, WINDOW</td>
   </tr>
   <tr>
      <td style="text-align: left;">[1.8 Last Salary within each department](#1.8-Last-Salary-within-each-department:)</td>
      <td style="text-align: left;">LAST, WINDOW</td>
   </tr>
   <tr>
      <td style="text-align: left;">[2.1 What are the best-selling and the second best-selling products in every category?](#2.1-What-are-the-best-selling-and-the-second-best-selling-products-in-every-category?)</td>
      <td style="text-align: left;">WINDOW, DENSE_RANK, INNER SUB-QUERY</td>
   </tr>
   <tr>
      <td style="text-align: left;">[2.2 What is the difference between the revenue of each product and the revenue of the best selling product in the same category as that product?](#2.2-What-is-the-difference-between-the-revenue-of-each-product-and-the-revenue-of-the-best-selling-product-in-the-same-category-as-that-product?)</td>
      <td style="text-align: left;">Highlights the Difference of ROWS BETWEEN and RANGE BETWEEN of WINDOW function</td>
   </tr>
   <tr>
      <td style="text-align: left;">[3.1 What is the first Purchase Date for each customer](#3.1-What-is-the-first-Purchase-Date-for-each-customer:)</td>
      <td style="text-align: left;">FIRST, WINDOW, INNER SUB-QUERY</td>
   </tr>
   <tr>
      <td style="text-align: left;">[3.2 What is the cumulative amount spent by each customer](#3.2-What-is-the-cumulative-amount-spent-by-each-customer:)</td>
      <td style="text-align: left;">SUM, WINDOW ROWS BETWEEN</td>
   </tr>
   <tr>
      <td style="text-align: left;">[3.3 What is the moving average between 3 consecutive transactions [previous, current, next] for each customer](#3.3-What-is-the-moving-average-between-3-consecutive-transactions-[previous,-current,-next]-for-each-customer:)</td>
      <td style="text-align: left;">SUM, WINDOW with specific PRECEDING AND FOLLOWING values for ROWS BETWEEN</td>
   </tr>
   <tr>
      <td style="text-align: left;">[3.4 What is the 3 days moving average for each customer](#3.4-What-is-the-3-days-moving-average-for-each-customer:)</td>
      <td style="text-align: left;">SUM, WINDOW with specific PRECEDING AND FOLLOWING values for RANGE BETWEEN</td>
   </tr>
   <tr>
      <td style="text-align: left;">[4.1 Running Difference of Current Sale from the Previous Sale](#4.1-Running-Difference-of-Current-Sale-from-the-Previous-Sale:)</td>
      <td style="text-align: left;">LAG, WINDOW</td>
   </tr>
   <tr>
      <td style="text-align: left;">[5.1 Forward-fill and Backward-fill missing data in Spark](#5.1-Forward-fill-and-Backward-fill-missing-data-in-Spark:)</td>
      <td style="text-align: left;">FIRST, LAST, WINDOW</td>
   </tr>
   <tr>
      <td style="text-align: left;">[5.2 Forward-fill and Backward-fill missing data without grouping
](#5.2-Forward-fill-and-Backward-fill-missing-data-without-grouping:)</td>
      <td style="text-align: left;">FIRST, LAST, WINDOW with no PARTITION BY</td>
   </tr>
   <tr>
      <td style="text-align: left;">[5.3 Forward-fill and Backward-fill missing data with a threshold](#5.3-Forward-fill-and-Backward-fill-missing-data-with-a-threshold:)</td>
      <td style="text-align: left;">FIRST, LAST, WINDOW with no specific ROWS BETWEEN for PRECEDING and FOLLOWING</td>
   </tr>
</table>

In [1]:
import os
import pandas as pd
import numpy as np

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext

from pyspark.sql.types import *
from pyspark.sql.window import Window

import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col

In [2]:
# setting random seed for notebook reproducability
rnd_seed=23
np.random.seed=23
np.random.set_state=23

In [3]:
# The following must be set in your .bashrc file
#SPARK_HOME="/home/ubuntu/spark-2.4.0-bin-hadoop2.7"
#ANACONDA_HOME="/home/ubuntu/anaconda3/envs/pyspark"
#PYSPARK_PYTHON="$ANACONDA_HOME/bin/python"
#PYSPARK_DRIVER_PYTHON="$ANACONDA_HOME/bin/python"
#PYTHONPATH="$ANACONDA_HOME/bin/python"
#export PATH="$ANACONDA_HOME/bin:$SPARK_HOME/bin:$PATH"

In [4]:
spark = (SparkSession
         .builder
         .master("local[*]")
         .appName("window-functions-with-pyspark")
         .getOrCreate())

In [5]:
spark

In [6]:
sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=window-functions-with-pyspark>

In [7]:
sqlContext = SQLContext(spark.sparkContext)
sqlContext

In [8]:
import re

# Utility function to emulate stripMargin in Scala string.
def strip_margin(text):
    nomargin = re.sub('\n[ \t]*\|', ' ', text)
    trimmed = re.sub('\s+', ' ', nomargin)
    return trimmed

## 1. Employee Data

In [9]:
# Create Sample Dataframe
employees = [
    (7369, "SMITH", "CLERK", 7902, "17-Dec-80", 800, 20, 10),
    (7499, "ALLEN", "SALESMAN", 7698, "20-Feb-81", 1600, 300, 30),
    (7521, "WARD", "SALESMAN", 7698, "22-Feb-81", 1250, 500, 30),
    (7566, "JONES", "MANAGER", 7839, "2-Apr-81", 2975, 0, 20),
    (7654, "MARTIN", "SALESMAN", 7698, "28-Sep-81", 1250, 1400, 30),
    (7698, "BLAKE", "MANAGER", 7839, "1-May-81", 2850, 0, 30),
    (7782, "CLARK", "MANAGER", 7839, "9-Jun-81", 2450, 0, 10),
    (7788, "SCOTT", "ANALYST", 7566, "19-Apr-87", 3000, 0, 20),
    (7629, "ALEX", "SALESMAN", 7698, "28-Sep-79", 1150, 1400, 30),
    (7839, "KING", "PRESIDENT", 0, "17-Nov-81", 5000, 0, 10),
    (7844, "TURNER", "SALESMAN", 7698, "8-Sep-81", 1500, 0, 30),
    (7876, "ADAMS", "CLERK", 7788, "23-May-87", 1100, 0, 20)    
]

In [10]:
emp_df = spark.createDataFrame(employees, ["empno", "ename", "job", "mgr", "hiredate", "sal", "comm", "deptno"])

In [11]:
emp_df.show(20)

+-----+------+---------+----+---------+----+----+------+
|empno| ename|      job| mgr| hiredate| sal|comm|deptno|
+-----+------+---------+----+---------+----+----+------+
| 7369| SMITH|    CLERK|7902|17-Dec-80| 800|  20|    10|
| 7499| ALLEN| SALESMAN|7698|20-Feb-81|1600| 300|    30|
| 7521|  WARD| SALESMAN|7698|22-Feb-81|1250| 500|    30|
| 7566| JONES|  MANAGER|7839| 2-Apr-81|2975|   0|    20|
| 7654|MARTIN| SALESMAN|7698|28-Sep-81|1250|1400|    30|
| 7698| BLAKE|  MANAGER|7839| 1-May-81|2850|   0|    30|
| 7782| CLARK|  MANAGER|7839| 9-Jun-81|2450|   0|    10|
| 7788| SCOTT|  ANALYST|7566|19-Apr-87|3000|   0|    20|
| 7629|  ALEX| SALESMAN|7698|28-Sep-79|1150|1400|    30|
| 7839|  KING|PRESIDENT|   0|17-Nov-81|5000|   0|    10|
| 7844|TURNER| SALESMAN|7698| 8-Sep-81|1500|   0|    30|
| 7876| ADAMS|    CLERK|7788|23-May-87|1100|   0|    20|
+-----+------+---------+----+---------+----+----+------+



In [12]:
emp_df.printSchema()

root
 |-- empno: long (nullable = true)
 |-- ename: string (nullable = true)
 |-- job: string (nullable = true)
 |-- mgr: long (nullable = true)
 |-- hiredate: string (nullable = true)
 |-- sal: long (nullable = true)
 |-- comm: long (nullable = true)
 |-- deptno: long (nullable = true)



In [13]:
emp_df.createOrReplaceTempView("emp")

### 1.1 Rank salary within each department:

**SQL Using rank() Window Function:**

In [14]:
spark.sql(strip_margin(
        """SELECT empno, ename, job, deptno, sal, RANK() OVER (PARTITION BY deptno ORDER BY sal desc) AS rank
          |FROM emp
        """)).show(20)

+-----+------+---------+------+----+----+
|empno| ename|      job|deptno| sal|rank|
+-----+------+---------+------+----+----+
| 7839|  KING|PRESIDENT|    10|5000|   1|
| 7782| CLARK|  MANAGER|    10|2450|   2|
| 7369| SMITH|    CLERK|    10| 800|   3|
| 7698| BLAKE|  MANAGER|    30|2850|   1|
| 7499| ALLEN| SALESMAN|    30|1600|   2|
| 7844|TURNER| SALESMAN|    30|1500|   3|
| 7521|  WARD| SALESMAN|    30|1250|   4|
| 7654|MARTIN| SALESMAN|    30|1250|   4|
| 7629|  ALEX| SALESMAN|    30|1150|   6|
| 7788| SCOTT|  ANALYST|    20|3000|   1|
| 7566| JONES|  MANAGER|    20|2975|   2|
| 7876| ADAMS|    CLERK|    20|1100|   3|
+-----+------+---------+------+----+----+



**DF API Using Window Function:**

First of all we will need to define the window we will be working on i.e. we will partition by department (deptno) and order by salary (sal).

In [15]:
windowSpec = Window.partitionBy(col('deptno')).orderBy(col('sal').desc())

In [16]:
emp_df.select('empno', 'ename', 'job', 'deptno', 'sal', F.rank().over(windowSpec).alias('rank')).show(20)

+-----+------+---------+------+----+----+
|empno| ename|      job|deptno| sal|rank|
+-----+------+---------+------+----+----+
| 7839|  KING|PRESIDENT|    10|5000|   1|
| 7782| CLARK|  MANAGER|    10|2450|   2|
| 7369| SMITH|    CLERK|    10| 800|   3|
| 7698| BLAKE|  MANAGER|    30|2850|   1|
| 7499| ALLEN| SALESMAN|    30|1600|   2|
| 7844|TURNER| SALESMAN|    30|1500|   3|
| 7521|  WARD| SALESMAN|    30|1250|   4|
| 7654|MARTIN| SALESMAN|    30|1250|   4|
| 7629|  ALEX| SALESMAN|    30|1150|   6|
| 7788| SCOTT|  ANALYST|    20|3000|   1|
| 7566| JONES|  MANAGER|    20|2975|   2|
| 7876| ADAMS|    CLERK|    20|1100|   3|
+-----+------+---------+------+----+----+



[Back to Top](#Table-of-contents)

### 1.2 Dense Rank salary within each department:

**SQL Using dense_rank() Window Function:**

In [17]:
spark.sql(strip_margin(
        """SELECT empno, ename, job, deptno, sal, DENSE_RANK() OVER (PARTITION BY deptno ORDER BY sal desc) AS dense_rank
          |FROM emp
        """)).show()

+-----+------+---------+------+----+----------+
|empno| ename|      job|deptno| sal|dense_rank|
+-----+------+---------+------+----+----------+
| 7839|  KING|PRESIDENT|    10|5000|         1|
| 7782| CLARK|  MANAGER|    10|2450|         2|
| 7369| SMITH|    CLERK|    10| 800|         3|
| 7698| BLAKE|  MANAGER|    30|2850|         1|
| 7499| ALLEN| SALESMAN|    30|1600|         2|
| 7844|TURNER| SALESMAN|    30|1500|         3|
| 7521|  WARD| SALESMAN|    30|1250|         4|
| 7654|MARTIN| SALESMAN|    30|1250|         4|
| 7629|  ALEX| SALESMAN|    30|1150|         5|
| 7788| SCOTT|  ANALYST|    20|3000|         1|
| 7566| JONES|  MANAGER|    20|2975|         2|
| 7876| ADAMS|    CLERK|    20|1100|         3|
+-----+------+---------+------+----+----------+



**DF API Using Window Function:**

In [18]:
windowSpec = Window.partitionBy(col('deptno')).orderBy(col('sal').desc())

In [19]:
emp_df.select('empno', 'ename', 'job', 'deptno', 'sal', F.dense_rank().over(windowSpec).alias('dense_rank')).show()

+-----+------+---------+------+----+----------+
|empno| ename|      job|deptno| sal|dense_rank|
+-----+------+---------+------+----+----------+
| 7839|  KING|PRESIDENT|    10|5000|         1|
| 7782| CLARK|  MANAGER|    10|2450|         2|
| 7369| SMITH|    CLERK|    10| 800|         3|
| 7698| BLAKE|  MANAGER|    30|2850|         1|
| 7499| ALLEN| SALESMAN|    30|1600|         2|
| 7844|TURNER| SALESMAN|    30|1500|         3|
| 7521|  WARD| SALESMAN|    30|1250|         4|
| 7654|MARTIN| SALESMAN|    30|1250|         4|
| 7629|  ALEX| SALESMAN|    30|1150|         5|
| 7788| SCOTT|  ANALYST|    20|3000|         1|
| 7566| JONES|  MANAGER|    20|2975|         2|
| 7876| ADAMS|    CLERK|    20|1100|         3|
+-----+------+---------+------+----+----------+



[Back to Top](#Table-of-contents)

### 1.3 Row Number within each department:

**SQL Using row_num() Window Function:**

In [20]:
spark.sql(strip_margin(
        """SELECT empno, ename, job, deptno, sal, ROW_NUMBER() OVER (PARTITION BY deptno ORDER BY sal desc) AS row_num
          |FROM emp
        """)).show()

+-----+------+---------+------+----+-------+
|empno| ename|      job|deptno| sal|row_num|
+-----+------+---------+------+----+-------+
| 7839|  KING|PRESIDENT|    10|5000|      1|
| 7782| CLARK|  MANAGER|    10|2450|      2|
| 7369| SMITH|    CLERK|    10| 800|      3|
| 7698| BLAKE|  MANAGER|    30|2850|      1|
| 7499| ALLEN| SALESMAN|    30|1600|      2|
| 7844|TURNER| SALESMAN|    30|1500|      3|
| 7521|  WARD| SALESMAN|    30|1250|      4|
| 7654|MARTIN| SALESMAN|    30|1250|      5|
| 7629|  ALEX| SALESMAN|    30|1150|      6|
| 7788| SCOTT|  ANALYST|    20|3000|      1|
| 7566| JONES|  MANAGER|    20|2975|      2|
| 7876| ADAMS|    CLERK|    20|1100|      3|
+-----+------+---------+------+----+-------+



**DF API Using Window Function:**

In [21]:
windowSpec = Window.partitionBy(col('deptno')).orderBy(col('sal').desc())

In [22]:
emp_df.select('empno', 'ename', 'job', 'deptno', 'sal', F.row_number().over(windowSpec).alias('row_num')).show()

+-----+------+---------+------+----+-------+
|empno| ename|      job|deptno| sal|row_num|
+-----+------+---------+------+----+-------+
| 7839|  KING|PRESIDENT|    10|5000|      1|
| 7782| CLARK|  MANAGER|    10|2450|      2|
| 7369| SMITH|    CLERK|    10| 800|      3|
| 7698| BLAKE|  MANAGER|    30|2850|      1|
| 7499| ALLEN| SALESMAN|    30|1600|      2|
| 7844|TURNER| SALESMAN|    30|1500|      3|
| 7521|  WARD| SALESMAN|    30|1250|      4|
| 7654|MARTIN| SALESMAN|    30|1250|      5|
| 7629|  ALEX| SALESMAN|    30|1150|      6|
| 7788| SCOTT|  ANALYST|    20|3000|      1|
| 7566| JONES|  MANAGER|    20|2975|      2|
| 7876| ADAMS|    CLERK|    20|1100|      3|
+-----+------+---------+------+----+-------+



[Back to Top](#Table-of-contents)

### 1.4 Running Total Salary within each department:

**SQL Using sum() with Window Function:**

In [23]:
spark.sql(strip_margin(
        """SELECT empno, ename, job, deptno, sal, SUM(sal) OVER (PARTITION BY deptno ORDER BY sal desc) AS running_total
          |FROM emp
        """)).show()

+-----+------+---------+------+----+-------------+
|empno| ename|      job|deptno| sal|running_total|
+-----+------+---------+------+----+-------------+
| 7839|  KING|PRESIDENT|    10|5000|         5000|
| 7782| CLARK|  MANAGER|    10|2450|         7450|
| 7369| SMITH|    CLERK|    10| 800|         8250|
| 7698| BLAKE|  MANAGER|    30|2850|         2850|
| 7499| ALLEN| SALESMAN|    30|1600|         4450|
| 7844|TURNER| SALESMAN|    30|1500|         5950|
| 7521|  WARD| SALESMAN|    30|1250|         8450|
| 7654|MARTIN| SALESMAN|    30|1250|         8450|
| 7629|  ALEX| SALESMAN|    30|1150|         9600|
| 7788| SCOTT|  ANALYST|    20|3000|         3000|
| 7566| JONES|  MANAGER|    20|2975|         5975|
| 7876| ADAMS|    CLERK|    20|1100|         7075|
+-----+------+---------+------+----+-------------+



**DF API Using sum() with Window Function:**

In [24]:
windowSpec = Window.partitionBy(col('deptno')).orderBy(col('sal').desc())

In [25]:
emp_df.select('empno', 'ename', 'job', 'deptno', 'sal', F.sum('sal').over(windowSpec).alias('running_total')).show()

+-----+------+---------+------+----+-------------+
|empno| ename|      job|deptno| sal|running_total|
+-----+------+---------+------+----+-------------+
| 7839|  KING|PRESIDENT|    10|5000|         5000|
| 7782| CLARK|  MANAGER|    10|2450|         7450|
| 7369| SMITH|    CLERK|    10| 800|         8250|
| 7698| BLAKE|  MANAGER|    30|2850|         2850|
| 7499| ALLEN| SALESMAN|    30|1600|         4450|
| 7844|TURNER| SALESMAN|    30|1500|         5950|
| 7521|  WARD| SALESMAN|    30|1250|         8450|
| 7654|MARTIN| SALESMAN|    30|1250|         8450|
| 7629|  ALEX| SALESMAN|    30|1150|         9600|
| 7788| SCOTT|  ANALYST|    20|3000|         3000|
| 7566| JONES|  MANAGER|    20|2975|         5975|
| 7876| ADAMS|    CLERK|    20|1100|         7075|
+-----+------+---------+------+----+-------------+



In [26]:
windowSpec = Window.partitionBy(col('deptno')).orderBy(col('sal').desc()).rowsBetween(Window.unboundedPreceding, Window.currentRow)

In [27]:
emp_df.select('empno', 'ename', 'job', 'deptno', 'sal', F.round(F.avg('sal').over(windowSpec), 2).alias('moving_avg')).show()

+-----+------+---------+------+----+----------+
|empno| ename|      job|deptno| sal|moving_avg|
+-----+------+---------+------+----+----------+
| 7839|  KING|PRESIDENT|    10|5000|    5000.0|
| 7782| CLARK|  MANAGER|    10|2450|    3725.0|
| 7369| SMITH|    CLERK|    10| 800|    2750.0|
| 7698| BLAKE|  MANAGER|    30|2850|    2850.0|
| 7499| ALLEN| SALESMAN|    30|1600|    2225.0|
| 7844|TURNER| SALESMAN|    30|1500|   1983.33|
| 7521|  WARD| SALESMAN|    30|1250|    1800.0|
| 7654|MARTIN| SALESMAN|    30|1250|    1690.0|
| 7629|  ALEX| SALESMAN|    30|1150|    1600.0|
| 7788| SCOTT|  ANALYST|    20|3000|    3000.0|
| 7566| JONES|  MANAGER|    20|2975|    2987.5|
| 7876| ADAMS|    CLERK|    20|1100|   2358.33|
+-----+------+---------+------+----+----------+



[Back to Top](#Table-of-contents)

### 1.5 Next Salary within each department:
Lead function allows us to compare current row with subsequent rows within each partition depending on the second argument (offset) which is by default set to 1 i.e. next row but we can change that parameter 2 to compare against every other row. The 3rd parameter is default value to be returned when no subsequent values exists or null.

**SQL Using lead() with Window Function:**

In [28]:
spark.sql(strip_margin(
        """SELECT empno, ename, job, deptno, sal, LEAD(sal, 1) OVER (PARTITION BY deptno ORDER BY sal desc) AS next_val
          |FROM emp
        """)).show()

+-----+------+---------+------+----+--------+
|empno| ename|      job|deptno| sal|next_val|
+-----+------+---------+------+----+--------+
| 7839|  KING|PRESIDENT|    10|5000|    2450|
| 7782| CLARK|  MANAGER|    10|2450|     800|
| 7369| SMITH|    CLERK|    10| 800|    null|
| 7698| BLAKE|  MANAGER|    30|2850|    1600|
| 7499| ALLEN| SALESMAN|    30|1600|    1500|
| 7844|TURNER| SALESMAN|    30|1500|    1250|
| 7521|  WARD| SALESMAN|    30|1250|    1250|
| 7654|MARTIN| SALESMAN|    30|1250|    1150|
| 7629|  ALEX| SALESMAN|    30|1150|    null|
| 7788| SCOTT|  ANALYST|    20|3000|    2975|
| 7566| JONES|  MANAGER|    20|2975|    1100|
| 7876| ADAMS|    CLERK|    20|1100|    null|
+-----+------+---------+------+----+--------+



**DF API Using lead() with Window Function:**

In [29]:
windowSpec = Window.partitionBy(col('deptno')).orderBy(col('sal').desc())

In [30]:
emp_df.select('empno', 'ename', 'job', 'deptno', 'sal', F.lead('sal', count=1, default=0).over(windowSpec).alias('next_val')).show()

+-----+------+---------+------+----+--------+
|empno| ename|      job|deptno| sal|next_val|
+-----+------+---------+------+----+--------+
| 7839|  KING|PRESIDENT|    10|5000|    2450|
| 7782| CLARK|  MANAGER|    10|2450|     800|
| 7369| SMITH|    CLERK|    10| 800|       0|
| 7698| BLAKE|  MANAGER|    30|2850|    1600|
| 7499| ALLEN| SALESMAN|    30|1600|    1500|
| 7844|TURNER| SALESMAN|    30|1500|    1250|
| 7521|  WARD| SALESMAN|    30|1250|    1250|
| 7654|MARTIN| SALESMAN|    30|1250|    1150|
| 7629|  ALEX| SALESMAN|    30|1150|       0|
| 7788| SCOTT|  ANALYST|    20|3000|    2975|
| 7566| JONES|  MANAGER|    20|2975|    1100|
| 7876| ADAMS|    CLERK|    20|1100|       0|
+-----+------+---------+------+----+--------+



[Back to Top](#Table-of-contents)

### 1.6 Previous Salary within each department:
Lag function allows us to compare current row with preceding rows within each partition depending on the second argument (offset) which is by default set to 1 i.e. next row but we can change that parameter 2 to compare against every other row. The 3rd parameter is default value to be returned when no subsequent values exists or null.

Accesses data from a previous row in the same result set without the use of a self-join starting with SQL Server 2012. LAG provides access to a row at a given physical offset that comes before the current row. Use this analytic function in a SELECT statement to compare values in the current row with values in a previous row.

**SQL Using lag() with Window Function:**

In [31]:
spark.sql(strip_margin(
        """SELECT empno, ename, job, deptno, sal, LAG(sal, 1) OVER (PARTITION BY deptno ORDER BY sal desc) AS prev_val
          |FROM emp
        """)).show()

+-----+------+---------+------+----+--------+
|empno| ename|      job|deptno| sal|prev_val|
+-----+------+---------+------+----+--------+
| 7839|  KING|PRESIDENT|    10|5000|    null|
| 7782| CLARK|  MANAGER|    10|2450|    5000|
| 7369| SMITH|    CLERK|    10| 800|    2450|
| 7698| BLAKE|  MANAGER|    30|2850|    null|
| 7499| ALLEN| SALESMAN|    30|1600|    2850|
| 7844|TURNER| SALESMAN|    30|1500|    1600|
| 7521|  WARD| SALESMAN|    30|1250|    1500|
| 7654|MARTIN| SALESMAN|    30|1250|    1250|
| 7629|  ALEX| SALESMAN|    30|1150|    1250|
| 7788| SCOTT|  ANALYST|    20|3000|    null|
| 7566| JONES|  MANAGER|    20|2975|    3000|
| 7876| ADAMS|    CLERK|    20|1100|    2975|
+-----+------+---------+------+----+--------+



**DF API Using lag() with Window Function:**

In [32]:
windowSpec = Window.partitionBy(col('deptno')).orderBy(col('sal').desc())

In [33]:
emp_df.select('empno', 'ename', 'job', 'deptno', 'sal', F.lag('sal', count=1, default=0).over(windowSpec).alias('prev_val')).show()

+-----+------+---------+------+----+--------+
|empno| ename|      job|deptno| sal|prev_val|
+-----+------+---------+------+----+--------+
| 7839|  KING|PRESIDENT|    10|5000|       0|
| 7782| CLARK|  MANAGER|    10|2450|    5000|
| 7369| SMITH|    CLERK|    10| 800|    2450|
| 7698| BLAKE|  MANAGER|    30|2850|       0|
| 7499| ALLEN| SALESMAN|    30|1600|    2850|
| 7844|TURNER| SALESMAN|    30|1500|    1600|
| 7521|  WARD| SALESMAN|    30|1250|    1500|
| 7654|MARTIN| SALESMAN|    30|1250|    1250|
| 7629|  ALEX| SALESMAN|    30|1150|    1250|
| 7788| SCOTT|  ANALYST|    20|3000|       0|
| 7566| JONES|  MANAGER|    20|2975|    3000|
| 7876| ADAMS|    CLERK|    20|1100|    2975|
+-----+------+---------+------+----+--------+



[Back to Top](#Table-of-contents)

### 1.7 First Salary within each department:
First value within each partition .i.e. highest salary (we are using order by descending) within each department can be compared against every member within each department.

**SQL Using first() with Window Function:**

In [34]:
spark.sql(strip_margin(
        """SELECT empno, ename, job, deptno, sal, FIRST_VALUE(sal) OVER (PARTITION BY deptno ORDER BY sal desc) AS first_val
          |FROM emp
        """)).show()

+-----+------+---------+------+----+---------+
|empno| ename|      job|deptno| sal|first_val|
+-----+------+---------+------+----+---------+
| 7839|  KING|PRESIDENT|    10|5000|     5000|
| 7782| CLARK|  MANAGER|    10|2450|     5000|
| 7369| SMITH|    CLERK|    10| 800|     5000|
| 7698| BLAKE|  MANAGER|    30|2850|     2850|
| 7499| ALLEN| SALESMAN|    30|1600|     2850|
| 7844|TURNER| SALESMAN|    30|1500|     2850|
| 7521|  WARD| SALESMAN|    30|1250|     2850|
| 7654|MARTIN| SALESMAN|    30|1250|     2850|
| 7629|  ALEX| SALESMAN|    30|1150|     2850|
| 7788| SCOTT|  ANALYST|    20|3000|     3000|
| 7566| JONES|  MANAGER|    20|2975|     3000|
| 7876| ADAMS|    CLERK|    20|1100|     3000|
+-----+------+---------+------+----+---------+



**DF API Using first() with Window Function:**

In [35]:
windowSpec = Window.partitionBy(col('deptno')).orderBy(col('sal').desc())

In [36]:
emp_df.select('empno', 'ename', 'job', 'deptno', 'sal', F.first('sal').over(windowSpec).alias('first_val')).show()

+-----+------+---------+------+----+---------+
|empno| ename|      job|deptno| sal|first_val|
+-----+------+---------+------+----+---------+
| 7839|  KING|PRESIDENT|    10|5000|     5000|
| 7782| CLARK|  MANAGER|    10|2450|     5000|
| 7369| SMITH|    CLERK|    10| 800|     5000|
| 7698| BLAKE|  MANAGER|    30|2850|     2850|
| 7499| ALLEN| SALESMAN|    30|1600|     2850|
| 7844|TURNER| SALESMAN|    30|1500|     2850|
| 7521|  WARD| SALESMAN|    30|1250|     2850|
| 7654|MARTIN| SALESMAN|    30|1250|     2850|
| 7629|  ALEX| SALESMAN|    30|1150|     2850|
| 7788| SCOTT|  ANALYST|    20|3000|     3000|
| 7566| JONES|  MANAGER|    20|2975|     3000|
| 7876| ADAMS|    CLERK|    20|1100|     3000|
+-----+------+---------+------+----+---------+



[Back to Top](#Table-of-contents)

### 1.8 Last Salary within each department:
Last value within each partition .i.e. lowest salary (we are using order by descending) within each department can be compared against every member within each department.

**SQL Using last() with Window Function:**

In [37]:
spark.sql(strip_margin(
        """SELECT empno, ename, job, deptno, sal, LAST_VALUE(sal) OVER (PARTITION BY deptno ORDER BY sal desc) AS last_val
          |FROM emp
        """)).show()

+-----+------+---------+------+----+--------+
|empno| ename|      job|deptno| sal|last_val|
+-----+------+---------+------+----+--------+
| 7839|  KING|PRESIDENT|    10|5000|    5000|
| 7782| CLARK|  MANAGER|    10|2450|    2450|
| 7369| SMITH|    CLERK|    10| 800|     800|
| 7698| BLAKE|  MANAGER|    30|2850|    2850|
| 7499| ALLEN| SALESMAN|    30|1600|    1600|
| 7844|TURNER| SALESMAN|    30|1500|    1500|
| 7521|  WARD| SALESMAN|    30|1250|    1250|
| 7654|MARTIN| SALESMAN|    30|1250|    1250|
| 7629|  ALEX| SALESMAN|    30|1150|    1150|
| 7788| SCOTT|  ANALYST|    20|3000|    3000|
| 7566| JONES|  MANAGER|    20|2975|    2975|
| 7876| ADAMS|    CLERK|    20|1100|    1100|
+-----+------+---------+------+----+--------+



**Oops!** what happened here the `last_val` has the same value as in `sal` column but we were expecting the lowest salary within the department in the last_val column so for that we really need to understand how the window operates and works. There are two types of frames `ROW` and `RANGE`.The details are explained in this [posts](https://databricks.com/blog/2015/07/15/introducing-window-functions-in-spark-sql.html) from databricks.

*This happens because **default window frame is range between unbounded preceding and current row**, so the `last_value()` never looks beyond current row unless we change the frame.*

Last value fixed by supplying the window frame for `last_val()` to operate on. We will be using start frame current row and end frame unbounded following to get the last value.

In [38]:
spark.sql(strip_margin(
        """SELECT empno, ename, job, deptno, sal, 
          |    LAST_VALUE(sal) OVER (PARTITION BY deptno ORDER BY sal desc ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING) AS last_val
          |FROM emp
        """)).show()

+-----+------+---------+------+----+--------+
|empno| ename|      job|deptno| sal|last_val|
+-----+------+---------+------+----+--------+
| 7839|  KING|PRESIDENT|    10|5000|     800|
| 7782| CLARK|  MANAGER|    10|2450|     800|
| 7369| SMITH|    CLERK|    10| 800|     800|
| 7698| BLAKE|  MANAGER|    30|2850|    1150|
| 7499| ALLEN| SALESMAN|    30|1600|    1150|
| 7844|TURNER| SALESMAN|    30|1500|    1150|
| 7521|  WARD| SALESMAN|    30|1250|    1150|
| 7654|MARTIN| SALESMAN|    30|1250|    1150|
| 7629|  ALEX| SALESMAN|    30|1150|    1150|
| 7788| SCOTT|  ANALYST|    20|3000|    1100|
| 7566| JONES|  MANAGER|    20|2975|    1100|
| 7876| ADAMS|    CLERK|    20|1100|    1100|
+-----+------+---------+------+----+--------+



**DF API Using last() with Window Function:**

In [39]:
windowSpec = Window.partitionBy(col('deptno')).orderBy(col('sal').desc()).rowsBetween(Window.currentRow, Window.unboundedFollowing)

In [40]:
emp_df.select('empno', 'ename', 'job', 'deptno', 'sal', F.last('sal').over(windowSpec).alias('last_val')).show()

+-----+------+---------+------+----+--------+
|empno| ename|      job|deptno| sal|last_val|
+-----+------+---------+------+----+--------+
| 7839|  KING|PRESIDENT|    10|5000|     800|
| 7782| CLARK|  MANAGER|    10|2450|     800|
| 7369| SMITH|    CLERK|    10| 800|     800|
| 7698| BLAKE|  MANAGER|    30|2850|    1150|
| 7499| ALLEN| SALESMAN|    30|1600|    1150|
| 7844|TURNER| SALESMAN|    30|1500|    1150|
| 7521|  WARD| SALESMAN|    30|1250|    1150|
| 7654|MARTIN| SALESMAN|    30|1250|    1150|
| 7629|  ALEX| SALESMAN|    30|1150|    1150|
| 7788| SCOTT|  ANALYST|    20|3000|    1100|
| 7566| JONES|  MANAGER|    20|2975|    1100|
| 7876| ADAMS|    CLERK|    20|1100|    1100|
+-----+------+---------+------+----+--------+



[Back to Top](#Table-of-contents)

##  2. Product Sales Data

In [41]:
product_revenues = [
  ("Thin",       "cell phone", 6000),
  ("Normal",     "tablet",     1500),
  ("Mini",       "tablet",     5500),
  ("Ultra thin", "cell phone", 5000),
  ("Very thin",  "cell phone", 6000),
  ("Big",        "tablet",     2500),
  ("Bendable",   "cell phone", 3000),
  ("Foldable",   "cell phone", 3000),
  ("Pro",        "tablet",     4500),
  ("Pro2",       "tablet",     6500)
]

In [42]:
prod_rev_df = spark.createDataFrame(product_revenues, ["product", "category", "revenue"])

In [43]:
prod_rev_df.show(20)

+----------+----------+-------+
|   product|  category|revenue|
+----------+----------+-------+
|      Thin|cell phone|   6000|
|    Normal|    tablet|   1500|
|      Mini|    tablet|   5500|
|Ultra thin|cell phone|   5000|
| Very thin|cell phone|   6000|
|       Big|    tablet|   2500|
|  Bendable|cell phone|   3000|
|  Foldable|cell phone|   3000|
|       Pro|    tablet|   4500|
|      Pro2|    tablet|   6500|
+----------+----------+-------+



In [44]:
prod_rev_df.printSchema()

root
 |-- product: string (nullable = true)
 |-- category: string (nullable = true)
 |-- revenue: long (nullable = true)



In [45]:
prod_rev_df.createOrReplaceTempView("prod_rev")

[Back to Top](#Table-of-contents)

### 2.1 What are the best-selling and the second best-selling products in every category?

**SQL: Using dense_rank() with Window Function:**

In [46]:
spark.sql(strip_margin(
        """SELECT product, category, revenue, dense_rank
          |FROM (
          |    SELECT product, category, revenue, 
          |    DENSE_RANK(revenue) OVER (PARTITION BY category ORDER BY revenue desc) AS dense_rank
          |    FROM prod_rev ) a
          |WHERE dense_rank <= 2
        """)).show()

+----------+----------+-------+----------+
|   product|  category|revenue|dense_rank|
+----------+----------+-------+----------+
|      Pro2|    tablet|   6500|         1|
|      Mini|    tablet|   5500|         2|
|      Thin|cell phone|   6000|         1|
| Very thin|cell phone|   6000|         1|
|Ultra thin|cell phone|   5000|         2|
+----------+----------+-------+----------+



**DF API Using dense_rank() with Window Function:**

In [47]:
windowSpec = Window.partitionBy(col('category')).orderBy(col('revenue').desc())

In [48]:
(prod_rev_df
 .select('product', 'category', 'revenue', F.dense_rank().over(windowSpec).alias('dense_rank'))
 .where('dense_rank <= 2')
 .show())

+----------+----------+-------+----------+
|   product|  category|revenue|dense_rank|
+----------+----------+-------+----------+
|      Pro2|    tablet|   6500|         1|
|      Mini|    tablet|   5500|         2|
|      Thin|cell phone|   6000|         1|
| Very thin|cell phone|   6000|         1|
|Ultra thin|cell phone|   5000|         2|
+----------+----------+-------+----------+



[Back to Top](#Table-of-contents)

### 2.2 What is the difference between the revenue of each product and the revenue of the best selling product in the same category as that product?

A point to note here is that since we are only interested in the maximm revenue of each product within a category we need to scan over the entire rows within that category. So using either rangeBetween and rowsBetween will work the same in this case.

However, there is a difference between the `ROW` Frame and `RANGE` Frame:
`ROW` frames are based on physical offsets from the position of the current input row, which means that `CURRENT ROW`, `[value] PRECEDING`, or `[value] FOLLOWING` specifies a physical offset. If `CURRENT ROW` is used as a boundary, it represents the current input row. `[value] PRECEDING` and `[value] FOLLOWING` describes the number of rows appear before and after the current input row, respectively.

`RANGE` frames are based on logical offsets from the position of the current input row, and have similar syntax to the `ROW` frame. A logical offset is the difference between the value of the ordering expression of the current input row and the value of that same expression of the boundary row of the frame. Because of this definition, when a `RANGE` frame is used, only a single ordering expression is allowed. Also, for a RANGE frame, all rows having the same value of the ordering expression with the current input row are considered as same row as far as the boundary calculation is concerned.

This will be clarified with Customer Expense Dataset below.

**SQL: Using ROWS BETWEEN with Window Function:**

In [49]:
# we could have used RANGE BETWEEN as well
spark.sql(strip_margin(
        """SELECT product, category, revenue, 
          |    MAX(revenue) OVER (PARTITION BY category ORDER BY revenue desc ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) - revenue AS rev_diff 
          |FROM prod_rev
        """)).show()

+----------+----------+-------+--------+
|   product|  category|revenue|rev_diff|
+----------+----------+-------+--------+
|      Pro2|    tablet|   6500|       0|
|      Mini|    tablet|   5500|    1000|
|       Pro|    tablet|   4500|    2000|
|       Big|    tablet|   2500|    4000|
|    Normal|    tablet|   1500|    5000|
|      Thin|cell phone|   6000|       0|
| Very thin|cell phone|   6000|       0|
|Ultra thin|cell phone|   5000|    1000|
|  Bendable|cell phone|   3000|    3000|
|  Foldable|cell phone|   3000|    3000|
+----------+----------+-------+--------+



**DF API: Using rangeBetween() with Window Function:**

In [50]:
# we could have used .rowsBetween(...) as well
windowSpec = (Window
              .partitionBy(prod_rev_df['category'])
              .orderBy(prod_rev_df['revenue'].desc())
              .rangeBetween(Window.unboundedPreceding, Window.unboundedFollowing))

In [51]:
(prod_rev_df
 .select('product', 'category', 'revenue', 
             (F.max('revenue').over(windowSpec) - col('revenue')).alias('rev_diff'))
 .show())

+----------+----------+-------+--------+
|   product|  category|revenue|rev_diff|
+----------+----------+-------+--------+
|      Pro2|    tablet|   6500|       0|
|      Mini|    tablet|   5500|    1000|
|       Pro|    tablet|   4500|    2000|
|       Big|    tablet|   2500|    4000|
|    Normal|    tablet|   1500|    5000|
|      Thin|cell phone|   6000|       0|
| Very thin|cell phone|   6000|       0|
|Ultra thin|cell phone|   5000|    1000|
|  Bendable|cell phone|   3000|    3000|
|  Foldable|cell phone|   3000|    3000|
+----------+----------+-------+--------+



[Back to Top](#Table-of-contents)

## 3. Customer Expense Data

In this example dataset, there are three customers who have spent different amounts of money each day.

In [52]:
expense = [
    ('Alice', '05/01/2016', 50),
    ('Alice', '05/02/2016', 45),
    ('Alice', '05/02/2016', 14),
    ('Alice', '05/03/2016', 55),
    ('Alice', '05/04/2016', 47),
    ('Alice', '05/05/2016', 27),
    ('Alice', '05/05/2016', 19),
    ('Bob', '05/01/2016', 25),
    ('Bob', '05/01/2016', 56),
    ('Bob', '05/04/2016', 29),
    ('Bob', '05/09/2016', 27),
    ('Bob', '05/10/2016', 34),
    ('Bob', '05/11/2016', 67),
    ('Bob', '05/11/2016', 8),
    ('Bob', '05/12/2016', 32),
    ('Bob', '05/13/2016', 10),
    ('Charlie', '05/04/2016', 27),
    ('Charlie', '05/11/2016', 67)
]

In [53]:
expense_df = spark.createDataFrame(expense, ["name", "order_date", "amount_spent"])

In [54]:
# convert order_date into date format
expense_df = expense_df.withColumn('order_date', F.to_date(col("order_date"), 'MM/dd/yyyy'))

In [55]:
expense_df.show(20)

+-------+----------+------------+
|   name|order_date|amount_spent|
+-------+----------+------------+
|  Alice|2016-05-01|          50|
|  Alice|2016-05-02|          45|
|  Alice|2016-05-02|          14|
|  Alice|2016-05-03|          55|
|  Alice|2016-05-04|          47|
|  Alice|2016-05-05|          27|
|  Alice|2016-05-05|          19|
|    Bob|2016-05-01|          25|
|    Bob|2016-05-01|          56|
|    Bob|2016-05-04|          29|
|    Bob|2016-05-09|          27|
|    Bob|2016-05-10|          34|
|    Bob|2016-05-11|          67|
|    Bob|2016-05-11|           8|
|    Bob|2016-05-12|          32|
|    Bob|2016-05-13|          10|
|Charlie|2016-05-04|          27|
|Charlie|2016-05-11|          67|
+-------+----------+------------+



In [56]:
expense_df.createOrReplaceTempView("expense")

In [57]:
expense_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- order_date: date (nullable = true)
 |-- amount_spent: long (nullable = true)



### 3.1 What is the first Purchase Date for each customer:
We are interested in finding when Alice, Bob and Charlie made there first ever purchase? We can extend that to answer the question is the given purchase customer's first purchase or the customer has returned to purchase more?

**SQL: Using FIRST with Window Function:**

We use `FIRST` function in conjunction with `WINDOW` analytics to find the first purchase date for each customer. If the `first_order_date` is same as the `order_date` in the current row then this row is the customer's first transaction otherwise its a returning cutomer transaction.

In [58]:
spark.sql(strip_margin(
        """SELECT name, 
          |       order_date, 
          |       FIRST(order_date) OVER (PARTITION BY name ORDER BY order_date) AS first_order_date, 
          |       CASE 
          |           WHEN FIRST(order_date) OVER (PARTITION BY name ORDER BY order_date) = order_date then 0 else 1 
          |       END AS returning_customer
          |FROM expense
          |ORDER BY name, order_date
        """)).show()

+-------+----------+----------------+------------------+
|   name|order_date|first_order_date|returning_customer|
+-------+----------+----------------+------------------+
|  Alice|2016-05-01|      2016-05-01|                 0|
|  Alice|2016-05-02|      2016-05-01|                 1|
|  Alice|2016-05-02|      2016-05-01|                 1|
|  Alice|2016-05-03|      2016-05-01|                 1|
|  Alice|2016-05-04|      2016-05-01|                 1|
|  Alice|2016-05-05|      2016-05-01|                 1|
|  Alice|2016-05-05|      2016-05-01|                 1|
|    Bob|2016-05-01|      2016-05-01|                 0|
|    Bob|2016-05-01|      2016-05-01|                 0|
|    Bob|2016-05-04|      2016-05-01|                 1|
|    Bob|2016-05-09|      2016-05-01|                 1|
|    Bob|2016-05-10|      2016-05-01|                 1|
|    Bob|2016-05-11|      2016-05-01|                 1|
|    Bob|2016-05-11|      2016-05-01|                 1|
|    Bob|2016-05-12|      2016-

We can then filter on the `returning_customer` column to filter out only the first purchase date. We do `distinct` here because the customer can make more than one purchase on the first purchase date.

In [59]:
spark.sql(strip_margin(
        """SELECT DISTINCT name, order_date 
          |FROM(
          |    SELECT name, 
          |       order_date, 
          |       FIRST(order_date) OVER (PARTITION BY name ORDER BY order_date) AS first_order_date, 
          |       CASE 
          |           WHEN FIRST(order_date) OVER (PARTITION BY name ORDER BY order_date) = order_date then 0 else 1  
          |       END AS returning_customer
          |    FROM expense
          |    ) 
          |WHERE returning_customer = 0
          |ORDER BY name, order_date
        """)).show()

+-------+----------+
|   name|order_date|
+-------+----------+
|  Alice|2016-05-01|
|    Bob|2016-05-01|
|Charlie|2016-05-04|
+-------+----------+



**DF API: Using FIRST with Window Function:**

We can achieve the same thing using DataFrame APIs as well.

We use `FIRST` function in conjunction with `WINDOW` analytics to find the first purchase date for each customer. If the `first_order_date` is same as the `order_date` in the current row then this row is the customer's first transaction otherwise its a returning cutomer transaction.

In [60]:
windowSpec = (Window
              .partitionBy(expense_df['name'])
              .orderBy(expense_df['order_date']))

In [61]:
(expense_df
 .select('name', 
         'order_date', 
         F.first('order_date').over(windowSpec).alias('first_order_date'), 
         F.when(F.first('order_date').over(windowSpec) == col('order_date'), 0).otherwise(1).alias('returning_customer'))
 .orderBy('name', 'order_date')
 .show())

+-------+----------+----------------+------------------+
|   name|order_date|first_order_date|returning_customer|
+-------+----------+----------------+------------------+
|  Alice|2016-05-01|      2016-05-01|                 0|
|  Alice|2016-05-02|      2016-05-01|                 1|
|  Alice|2016-05-02|      2016-05-01|                 1|
|  Alice|2016-05-03|      2016-05-01|                 1|
|  Alice|2016-05-04|      2016-05-01|                 1|
|  Alice|2016-05-05|      2016-05-01|                 1|
|  Alice|2016-05-05|      2016-05-01|                 1|
|    Bob|2016-05-01|      2016-05-01|                 0|
|    Bob|2016-05-01|      2016-05-01|                 0|
|    Bob|2016-05-04|      2016-05-01|                 1|
|    Bob|2016-05-09|      2016-05-01|                 1|
|    Bob|2016-05-10|      2016-05-01|                 1|
|    Bob|2016-05-11|      2016-05-01|                 1|
|    Bob|2016-05-11|      2016-05-01|                 1|
|    Bob|2016-05-12|      2016-

We can then filter on the `returning_customer` column to filter out only the first purchase date. We do `distinct` here because the customer can make more than one purchase on the first purchase date.

In [62]:
(expense_df
 .select('name', 
         'order_date', 
         F.first('order_date').over(windowSpec).alias('first_order_date'), 
         F.when(F.first('order_date').over(windowSpec) == col('order_date'), 0).otherwise(1).alias('returning_customer'))
 .filter(col("returning_customer") == 0)
 .distinct()
 .select('name', 'order_date')
 .orderBy('name', 'order_date')
 .show())

+-------+----------+
|   name|order_date|
+-------+----------+
|  Alice|2016-05-01|
|    Bob|2016-05-01|
|Charlie|2016-05-04|
+-------+----------+



[Back to Top](#Table-of-contents)

### 3.2 What is the cumulative amount spent by each customer:

**SQL: Using ROWS BETWEEN with Window Function:**

In [63]:
spark.sql(strip_margin(
        """SELECT name, order_date, amount_spent, 
          |    ROUND(SUM(amount_spent) OVER (PARTITION BY name ORDER BY order_date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW), 2) AS cum_expense
          |FROM expense
          |ORDER BY name, order_date
        """)).show()

+-------+----------+------------+-----------+
|   name|order_date|amount_spent|cum_expense|
+-------+----------+------------+-----------+
|  Alice|2016-05-01|          50|         50|
|  Alice|2016-05-02|          45|         95|
|  Alice|2016-05-02|          14|        109|
|  Alice|2016-05-03|          55|        164|
|  Alice|2016-05-04|          47|        211|
|  Alice|2016-05-05|          27|        238|
|  Alice|2016-05-05|          19|        257|
|    Bob|2016-05-01|          25|         25|
|    Bob|2016-05-01|          56|         81|
|    Bob|2016-05-04|          29|        110|
|    Bob|2016-05-09|          27|        137|
|    Bob|2016-05-10|          34|        171|
|    Bob|2016-05-11|           8|        246|
|    Bob|2016-05-11|          67|        238|
|    Bob|2016-05-12|          32|        278|
|    Bob|2016-05-13|          10|        288|
|Charlie|2016-05-04|          27|         27|
|Charlie|2016-05-11|          67|         94|
+-------+----------+------------+-

**DF API: Using ROWS BETWEEN with Window Function:**

In [64]:
windowSpec = (Window
              .partitionBy(expense_df['name'])
              .orderBy(expense_df['order_date'])
              .rowsBetween(Window.unboundedPreceding, Window.currentRow))

In [65]:
(expense_df
 .select('name', 'order_date', 'amount_spent', F.round(F.sum('amount_spent').over(windowSpec), 2).alias('cum_expense'))
 .orderBy('name', 'order_date')
 .show())

+-------+----------+------------+-----------+
|   name|order_date|amount_spent|cum_expense|
+-------+----------+------------+-----------+
|  Alice|2016-05-01|          50|         50|
|  Alice|2016-05-02|          45|         95|
|  Alice|2016-05-02|          14|        109|
|  Alice|2016-05-03|          55|        164|
|  Alice|2016-05-04|          47|        211|
|  Alice|2016-05-05|          27|        238|
|  Alice|2016-05-05|          19|        257|
|    Bob|2016-05-01|          25|         25|
|    Bob|2016-05-01|          56|         81|
|    Bob|2016-05-04|          29|        110|
|    Bob|2016-05-09|          27|        137|
|    Bob|2016-05-10|          34|        171|
|    Bob|2016-05-11|           8|        246|
|    Bob|2016-05-11|          67|        238|
|    Bob|2016-05-12|          32|        278|
|    Bob|2016-05-13|          10|        288|
|Charlie|2016-05-04|          27|         27|
|Charlie|2016-05-11|          67|         94|
+-------+----------+------------+-

[Back to Top](#Table-of-contents)

### 3.3 What is the moving average between 3 consecutive transactions [previous, current, next] for each customer:

Note the difference between this and the next use case where we will see that the moving average of last three transactions is not as same as moving average of consecutive three days' transaction. So, we have to use ROWS BETWEEN and we cannot use RANGE BETWEEN because we have to stick to the bounds between previous and next row.

**SQL: Using ROWS BETWEEN with specific values with Window Function:**

In [66]:
spark.sql(strip_margin(
        """SELECT name, order_date, amount_spent, 
          |    ROUND(AVG(amount_spent) OVER (PARTITION BY name ORDER BY order_date ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING), 2) AS moving_avg
          |FROM expense
          |ORDER BY name, order_date
        """)).show()

+-------+----------+------------+----------+
|   name|order_date|amount_spent|moving_avg|
+-------+----------+------------+----------+
|  Alice|2016-05-01|          50|      47.5|
|  Alice|2016-05-02|          45|     36.33|
|  Alice|2016-05-02|          14|      38.0|
|  Alice|2016-05-03|          55|     38.67|
|  Alice|2016-05-04|          47|      43.0|
|  Alice|2016-05-05|          27|      31.0|
|  Alice|2016-05-05|          19|      23.0|
|    Bob|2016-05-01|          25|      40.5|
|    Bob|2016-05-01|          56|     36.67|
|    Bob|2016-05-04|          29|     37.33|
|    Bob|2016-05-09|          27|      30.0|
|    Bob|2016-05-10|          34|     42.67|
|    Bob|2016-05-11|           8|     35.67|
|    Bob|2016-05-11|          67|     36.33|
|    Bob|2016-05-12|          32|     16.67|
|    Bob|2016-05-13|          10|      21.0|
|Charlie|2016-05-04|          27|      47.0|
|Charlie|2016-05-11|          67|      47.0|
+-------+----------+------------+----------+



**DF API: Using ROWS BETWEEN with specific values with Window Function:**

In [67]:
windowSpec = (Window
              .partitionBy(expense_df['name'])
              .orderBy(expense_df['order_date'])
              .rowsBetween(-1, 1))

In [68]:
(expense_df
 .select('name', 'order_date', 'amount_spent', F.round(F.avg('amount_spent').over(windowSpec), 2).alias('moving_avg'))
 .orderBy('name', 'order_date')
 .show())

+-------+----------+------------+----------+
|   name|order_date|amount_spent|moving_avg|
+-------+----------+------------+----------+
|  Alice|2016-05-01|          50|      47.5|
|  Alice|2016-05-02|          45|     36.33|
|  Alice|2016-05-02|          14|      38.0|
|  Alice|2016-05-03|          55|     38.67|
|  Alice|2016-05-04|          47|      43.0|
|  Alice|2016-05-05|          27|      31.0|
|  Alice|2016-05-05|          19|      23.0|
|    Bob|2016-05-01|          25|      40.5|
|    Bob|2016-05-01|          56|     36.67|
|    Bob|2016-05-04|          29|     37.33|
|    Bob|2016-05-09|          27|      30.0|
|    Bob|2016-05-10|          34|     42.67|
|    Bob|2016-05-11|           8|     35.67|
|    Bob|2016-05-11|          67|     36.33|
|    Bob|2016-05-12|          32|     16.67|
|    Bob|2016-05-13|          10|      21.0|
|Charlie|2016-05-04|          27|      47.0|
|Charlie|2016-05-11|          67|      47.0|
+-------+----------+------------+----------+



[Back to Top](#Table-of-contents)

### 3.4 What is the 3 days moving average for each customer:

The difference between this and the previous use case is that there can be 1, 2, or N i.e. any number of transactions within past three days and not necessary only 3 transactions in last 3 days. So, we have to use RANGE BETWEEN and we cannot use ROWS BETWEEN because we do not know how many rows wil be there in last 3 days before the current day.

In [69]:
# convert order_date into long using typcasting
expense_df.select('order_date', col('order_date').cast('timestamp').cast('long').alias('order_timestamp')).show(5)

+----------+---------------+
|order_date|order_timestamp|
+----------+---------------+
|2016-05-01|     1462060800|
|2016-05-02|     1462147200|
|2016-05-02|     1462147200|
|2016-05-03|     1462233600|
|2016-05-04|     1462320000|
+----------+---------------+
only showing top 5 rows



In [70]:
# convert order_date into long using inbuilt pysparksql function unix_timestamp
expense_df.select('order_date', F.unix_timestamp(col('order_date')).alias('order_timestamp')).show(5)

+----------+---------------+
|order_date|order_timestamp|
+----------+---------------+
|2016-05-01|     1462060800|
|2016-05-02|     1462147200|
|2016-05-02|     1462147200|
|2016-05-03|     1462233600|
|2016-05-04|     1462320000|
+----------+---------------+
only showing top 5 rows



We can use the above trick to convert the order_date column into long and then compare with the 3 days previous timestamp long value.

**DF API: Using RANGE BETWEEN with specific values with Window Function:**

In [71]:
# a lambda function to return the prev_days * timestamp in long
days = lambda d: d * 86400

In [72]:
# Here we have to use
windowSpec = (Window
              .partitionBy(expense_df['name'])
              .orderBy(F.unix_timestamp(expense_df['order_date']))
              .rangeBetween(-days(2), 0))

In [73]:
(expense_df
 .select('name', 'order_date', F.date_sub('order_date', 2).alias('prev_3rd_day'), 'amount_spent', 
         F.round(F.avg('amount_spent').over(windowSpec), 2).alias('moving_avg'))
 .orderBy('name', 'order_date')
 .show())

+-------+----------+------------+------------+----------+
|   name|order_date|prev_3rd_day|amount_spent|moving_avg|
+-------+----------+------------+------------+----------+
|  Alice|2016-05-01|  2016-04-29|          50|      50.0|
|  Alice|2016-05-02|  2016-04-30|          45|     36.33|
|  Alice|2016-05-02|  2016-04-30|          14|     36.33|
|  Alice|2016-05-03|  2016-05-01|          55|      41.0|
|  Alice|2016-05-04|  2016-05-02|          47|     40.25|
|  Alice|2016-05-05|  2016-05-03|          19|      37.0|
|  Alice|2016-05-05|  2016-05-03|          27|      37.0|
|    Bob|2016-05-01|  2016-04-29|          56|      40.5|
|    Bob|2016-05-01|  2016-04-29|          25|      40.5|
|    Bob|2016-05-04|  2016-05-02|          29|      29.0|
|    Bob|2016-05-09|  2016-05-07|          27|      27.0|
|    Bob|2016-05-10|  2016-05-08|          34|      30.5|
|    Bob|2016-05-11|  2016-05-09|           8|      34.0|
|    Bob|2016-05-11|  2016-05-09|          67|      34.0|
|    Bob|2016-

Observe closely the moving_avg for 2nd and 3rd rows? So, while it was calculating for 2nd row it should have taken only amounts 50 and 45 and the average should have been 47.5 but **oops!** the average is 36.33.

Why is that so? The definition of Range Frames states that **Also, for a RANGE frame, all rows having the same value of the ordering expression with the current input row are considered as same row as far as the boundary calculation is concerned.**

So, while it was calculating for 2nd row it also took the 3rd row because the ordering expression is over order_date and the 2nd and 3rd rows have same order_date and they are considered same. Although, we restricted the window range between current row and previous three dates it still took the next row.

**SQL API: Using RANGE BETWEEN with specific values with Window Function:**

In [74]:
# 2 days in millis
2 * 86400

172800

In [75]:
spark.sql(strip_margin(
        """SELECT name, order_date, DATE_SUB(order_date, 3) AS prev_3rd_day, amount_spent, 
          |    ROUND(AVG(amount_spent) OVER (PARTITION BY name ORDER BY UNIX_TIMESTAMP(order_date) RANGE BETWEEN 172800 PRECEDING AND CURRENT ROW), 2) AS moving_avg
          |FROM expense
          |ORDER BY name, order_date
        """)).show()

+-------+----------+------------+------------+----------+
|   name|order_date|prev_3rd_day|amount_spent|moving_avg|
+-------+----------+------------+------------+----------+
|  Alice|2016-05-01|  2016-04-28|          50|      50.0|
|  Alice|2016-05-02|  2016-04-29|          45|     36.33|
|  Alice|2016-05-02|  2016-04-29|          14|     36.33|
|  Alice|2016-05-03|  2016-04-30|          55|      41.0|
|  Alice|2016-05-04|  2016-05-01|          47|     40.25|
|  Alice|2016-05-05|  2016-05-02|          19|      37.0|
|  Alice|2016-05-05|  2016-05-02|          27|      37.0|
|    Bob|2016-05-01|  2016-04-28|          25|      40.5|
|    Bob|2016-05-01|  2016-04-28|          56|      40.5|
|    Bob|2016-05-04|  2016-05-01|          29|      29.0|
|    Bob|2016-05-09|  2016-05-06|          27|      27.0|
|    Bob|2016-05-10|  2016-05-07|          34|      30.5|
|    Bob|2016-05-11|  2016-05-08|           8|      34.0|
|    Bob|2016-05-11|  2016-05-08|          67|      34.0|
|    Bob|2016-

[Back to Top](#Table-of-contents)

## 4. Tractor Sales Data

In [76]:
sales = [
    (1, 'Jan-03', 141),
    (2, 'Feb-03', 157),
    (3, 'Mar-03', 185),
    (4, 'Apr-03', 199),
    (5, 'May-03', 203),
    (6, 'Jun-03', 189),
    (7, 'Jul-03', 207),
    (8, 'Aug-03', 207),
    (9, 'Sep-03', 171),
    (10, 'Oct-03', 150),
    (11, 'Nov-03', 138),
    (12, 'Dec-03', 165),
    (13, 'Jan-04', 145),
    (14, 'Feb-04', 168),
    (15, 'Mar-04', 197),
    (16, 'Apr-04', 208),
    (17, 'May-04', 210),
    (18, 'Jun-04', 209),
    (19, 'Jul-04', 238),
    (20, 'Aug-04', 238)
]

In [77]:
sales_df = spark.createDataFrame(sales, ['tid', 'month_year', 'sales']).cache()

In [78]:
sales_df.printSchema()

root
 |-- tid: long (nullable = true)
 |-- month_year: string (nullable = true)
 |-- sales: long (nullable = true)



In [79]:
sales_df.createOrReplaceTempView("sales")

### 4.1 Running Difference of Current Sale from the Previous Sale:

**SQL: Using lag() with Window Function:**

In [80]:
spark.sql(strip_margin(
        """SELECT month_year, sales, 
          |    LAG(sales, 1) OVER (ORDER BY tid) AS sales_lag_1,
          |    LAG(sales, 2) OVER (ORDER BY tid) AS sales_lag_2,
          |    LAG(sales, 3) OVER (ORDER BY tid) AS sales_lag_3,
          |    sales - LAG(sales, 1) OVER (ORDER BY tid) AS diff_with_last_sales
          |FROM sales 
        """)).show()

+----------+-----+-----------+-----------+-----------+--------------------+
|month_year|sales|sales_lag_1|sales_lag_2|sales_lag_3|diff_with_last_sales|
+----------+-----+-----------+-----------+-----------+--------------------+
|    Jan-03|  141|       null|       null|       null|                null|
|    Feb-03|  157|        141|       null|       null|                  16|
|    Mar-03|  185|        157|        141|       null|                  28|
|    Apr-03|  199|        185|        157|        141|                  14|
|    May-03|  203|        199|        185|        157|                   4|
|    Jun-03|  189|        203|        199|        185|                 -14|
|    Jul-03|  207|        189|        203|        199|                  18|
|    Aug-03|  207|        207|        189|        203|                   0|
|    Sep-03|  171|        207|        207|        189|                 -36|
|    Oct-03|  150|        171|        207|        207|                 -21|
|    Nov-03|

**DF API: Using lag() with Window Function:**

In [81]:
windowSpec = Window.orderBy(sales_df['tid'])

In [82]:
(sales_df
 .select('month_year', 'sales', 
         F.lag('sales', count=1).over(windowSpec).alias('sales_lag_1'),
         F.lag('sales', count=2).over(windowSpec).alias('sales_lag_2'),
         F.lag('sales', count=3).over(windowSpec).alias('sales_lag_3'),
         (col('sales') - F.lag('sales', count=1).over(windowSpec)).alias('diff_with_last_sales'))
 .show())

+----------+-----+-----------+-----------+-----------+--------------------+
|month_year|sales|sales_lag_1|sales_lag_2|sales_lag_3|diff_with_last_sales|
+----------+-----+-----------+-----------+-----------+--------------------+
|    Jan-03|  141|       null|       null|       null|                null|
|    Feb-03|  157|        141|       null|       null|                  16|
|    Mar-03|  185|        157|        141|       null|                  28|
|    Apr-03|  199|        185|        157|        141|                  14|
|    May-03|  203|        199|        185|        157|                   4|
|    Jun-03|  189|        203|        199|        185|                 -14|
|    Jul-03|  207|        189|        203|        199|                  18|
|    Aug-03|  207|        207|        189|        203|                   0|
|    Sep-03|  171|        207|        207|        189|                 -36|
|    Oct-03|  150|        171|        207|        207|                 -21|
|    Nov-03|

[Back to Top](#Table-of-contents)

## 5. Weather Data

### 5.1 Forward-fill and Backward-fill missing data in Spark:

This section is inpired from [https://johnpaton.net/posts/forward-fill-spark/](https://johnpaton.net/posts/forward-fill-spark/)

Like the author, I faced the same problem while I started working with Apache Spark. I found that there is no forward fill and backward fill method in the Spark DataFrame just as in Pandas DataFrames. Such is the price of scalability. Since I was just starting with Spark so I applied groupBy on my Spark Dataframe then converted the grouped values of Spark Dataframe into pandas and then used the Pandas `.fillna` method, specifying specifying `method='ffill'` or `method='bfill'`. But once I stumbled on this blog this fell very well with the theme of this notebook.

**A word of caution:** The following approaches will collect all rows of each group to some executor node. This will result in failed jobs if the number of rows in some groups is larger than the memory of our executor nodes.

Imagine we are measuring the temperature in two spots in your back yard, one in the shade and one in the sun. We record a measurement every half hour so you can compare them. However, we got the cheapest possible digital thermometer, so a lot of the measurements end up missing. Our data may look something like this:

In [83]:
weather = [    
    ('2017-09-09 12:00:00', 'shade', 18.830184076113213),
    ('2017-09-09 12:01:00', 'sun',   None),
    ('2017-09-09 12:30:00', 'shade', None),
    ('2017-09-09 12:31:00', 'sun',   21.55237663805009),
    ('2017-09-09 13:00:00', 'shade', 18.59059750682235),
    ('2017-09-09 13:01:00', 'sun',   None),
    ('2017-09-09 13:30:00', 'shade', None),
    ('2017-09-09 13:31:00', 'sun',   22.587784977960474),
    ('2017-09-09 14:00:00', 'shade', 19.101003724324197),
    ('2017-09-09 14:01:00', 'sun',   20.548896316341516)
]

In [84]:
weather_df = spark.createDataFrame(weather, ['time', 'location', 'temp']).cache()

In [85]:
weather_df.show()

+-------------------+--------+------------------+
|               time|location|              temp|
+-------------------+--------+------------------+
|2017-09-09 12:00:00|   shade|18.830184076113213|
|2017-09-09 12:01:00|     sun|              null|
|2017-09-09 12:30:00|   shade|              null|
|2017-09-09 12:31:00|     sun| 21.55237663805009|
|2017-09-09 13:00:00|   shade| 18.59059750682235|
|2017-09-09 13:01:00|     sun|              null|
|2017-09-09 13:30:00|   shade|              null|
|2017-09-09 13:31:00|     sun|22.587784977960474|
|2017-09-09 14:00:00|   shade|19.101003724324197|
|2017-09-09 14:01:00|     sun|20.548896316341516|
+-------------------+--------+------------------+



In [86]:
weather_df.printSchema()

root
 |-- time: string (nullable = true)
 |-- location: string (nullable = true)
 |-- temp: double (nullable = true)



In [87]:
weather_df.createOrReplaceTempView("weather")

To compare the measurements each half hour (or maybe to do some machine learning), we need a way of filling in the missing measurements. If the value we are measuring (in this case temperature) changes slowly with respect to how frequently we make a measurement, then a forward fill may be a reasonable choice.

In Pandas, this is easy. We just do a groupby without aggregation, and to each group apply the `.fillna` method, specifying specifying `method='ffill'`:

`
df_filled = df.groupby('location').apply(lambda group: group.fillna(method='ffill'))
`

We can take help of the `pyspark.sql window` function `last` and `first`. As its name suggests, `last` returns the last value in the window (implying that the window must have a meaningful ordering). It takes an optional argument `ignorenulls` which, when set to True, causes `last` to return the last non-null value in the window, if such a value exists. The same logic applies for `first` except it returns the first non-null value in the window.

The strategy to forward fill in Spark is as follows. First we define a window, which is ordered in time, and which includes all the rows from the beginning of time up until the current row. We achieve this here simply by selecting the rows in the window as being the `rowsBetween Window.unboundedPreceding` (the largest negative value possible), and `Window.currentRow` (the current row). Specifying too large of a value for the rows doesn't cause any errors, so we can just use a very large number to be sure our window reaches until the very beginning of the dataframe. If we need to optimize memory usage, we can make your job much more efficient by finding the maximal number of consecutive nulls in our dataframe and only taking a large enough window to include all of those plus one non-null value. We partition the window by the `location` column to make sure that gaps only get filled with previous measurements from the same location.

We act with `last` over the window we have defined, specifying `ignorenulls=True`. If the current row is non-null, then the output will just be the value of current row. However, if the current row is null, then the function will return the most recent (last) non-null value in the window.

Similarly the strategy to backward fill in Spark is as follows. First we define a window, which is ordered in time, and which includes all the rows from the current row to up until the end of time. We achieve this here simply by selecting the rows in the window as being the `rowsBetween Window.currentRow` (the current row) and `Window.unboundedFollowing` (the largest positive value possible).

**DF API: Using last(), first() with Window Function:**

In [88]:
# define the window
fwd_window = (Window.partitionBy('location')
               .orderBy('time')
               .rowsBetween(Window.unboundedPreceding, Window.currentRow))

bckwd_window = (Window.partitionBy('location')
               .orderBy('time')
               .rowsBetween(Window.currentRow, Window.unboundedFollowing))

In [89]:
# define the forward-filled column
ffilled_column = F.last(weather_df['temp'], ignorenulls=True).over(fwd_window) # True: fill with last non-null
bfilled_column = F.first(weather_df['temp'], ignorenulls=True).over(bckwd_window) # True: fill with first non-null

In [90]:
# do the fill
weather_df_filled = weather_df.withColumn('temp_fwd_filled', ffilled_column).withColumn('temp_bckwd_filled', bfilled_column)

In [91]:
# show
weather_df_filled.orderBy('location', 'time').show()  

+-------------------+--------+------------------+------------------+------------------+
|               time|location|              temp|   temp_fwd_filled| temp_bckwd_filled|
+-------------------+--------+------------------+------------------+------------------+
|2017-09-09 12:00:00|   shade|18.830184076113213|18.830184076113213|18.830184076113213|
|2017-09-09 12:30:00|   shade|              null|18.830184076113213| 18.59059750682235|
|2017-09-09 13:00:00|   shade| 18.59059750682235| 18.59059750682235| 18.59059750682235|
|2017-09-09 13:30:00|   shade|              null| 18.59059750682235|19.101003724324197|
|2017-09-09 14:00:00|   shade|19.101003724324197|19.101003724324197|19.101003724324197|
|2017-09-09 12:01:00|     sun|              null|              null| 21.55237663805009|
|2017-09-09 12:31:00|     sun| 21.55237663805009| 21.55237663805009| 21.55237663805009|
|2017-09-09 13:01:00|     sun|              null| 21.55237663805009|22.587784977960474|
|2017-09-09 13:31:00|     sun|22

We can clearly see all the null values have been forward filled with the last known non-null values and likewise for backward filling. Note that one row for forward fill is still null after forward filling because that was the first row in that group and it was null itself. It had no row to look back.

**SQL: Using LAST_VALUE(), FIRST_VALUE() with Window Function:**

In [92]:
spark.sql(strip_margin(
        """SELECT time, location, temp, 
          |    LAST_VALUE(temp, TRUE) OVER (PARTITION BY location ORDER BY time ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS temp_fwd_filled,
          |    FIRST_VALUE(temp, TRUE) OVER (PARTITION BY location ORDER BY time ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING) AS temp_bckd_filled
          |FROM weather
          |ORDER BY location, time
        """)).show()

+-------------------+--------+------------------+------------------+------------------+
|               time|location|              temp|   temp_fwd_filled|  temp_bckd_filled|
+-------------------+--------+------------------+------------------+------------------+
|2017-09-09 12:00:00|   shade|18.830184076113213|18.830184076113213|18.830184076113213|
|2017-09-09 12:30:00|   shade|              null|18.830184076113213| 18.59059750682235|
|2017-09-09 13:00:00|   shade| 18.59059750682235| 18.59059750682235| 18.59059750682235|
|2017-09-09 13:30:00|   shade|              null| 18.59059750682235|19.101003724324197|
|2017-09-09 14:00:00|   shade|19.101003724324197|19.101003724324197|19.101003724324197|
|2017-09-09 12:01:00|     sun|              null|              null| 21.55237663805009|
|2017-09-09 12:31:00|     sun| 21.55237663805009| 21.55237663805009| 21.55237663805009|
|2017-09-09 13:01:00|     sun|              null| 21.55237663805009|22.587784977960474|
|2017-09-09 13:31:00|     sun|22

[Back to Top](#Table-of-contents)

### 5.2 Forward-fill and Backward-fill missing data without grouping:

If the data is a univariate time-series and we need to fill missing values by forward or backward filling then we do not need to do any grouping and just use the `orderBy` in the `Window` fucntion. However, this would bring all the data into into one partition in an executor. If the data set is large than can be hold in executor memory it wil result in OOM.

**DF API: Using last(), first() with Window Function:**

In [93]:
# define the window
fwd_window = (Window.orderBy('time')
               .rowsBetween(Window.unboundedPreceding, Window.currentRow))

bckwd_window = (Window.orderBy('time')
               .rowsBetween(Window.currentRow, Window.unboundedFollowing))

In [94]:
# define the forward-filled column
ffilled_column = F.last(weather_df['temp'], ignorenulls=True).over(fwd_window) # True: fill with last non-null
bfilled_column = F.first(weather_df['temp'], ignorenulls=True).over(bckwd_window) # True: fill with first non-null

In [95]:
# do the fill
weather_df_filled = weather_df.withColumn('temp_fwd_filled', ffilled_column).withColumn('temp_bckwd_filled', bfilled_column)

In [96]:
# show
weather_df_filled.orderBy('time').show()  

+-------------------+--------+------------------+------------------+------------------+
|               time|location|              temp|   temp_fwd_filled| temp_bckwd_filled|
+-------------------+--------+------------------+------------------+------------------+
|2017-09-09 12:00:00|   shade|18.830184076113213|18.830184076113213|18.830184076113213|
|2017-09-09 12:01:00|     sun|              null|18.830184076113213| 21.55237663805009|
|2017-09-09 12:30:00|   shade|              null|18.830184076113213| 21.55237663805009|
|2017-09-09 12:31:00|     sun| 21.55237663805009| 21.55237663805009| 21.55237663805009|
|2017-09-09 13:00:00|   shade| 18.59059750682235| 18.59059750682235| 18.59059750682235|
|2017-09-09 13:01:00|     sun|              null| 18.59059750682235|22.587784977960474|
|2017-09-09 13:30:00|   shade|              null| 18.59059750682235|22.587784977960474|
|2017-09-09 13:31:00|     sun|22.587784977960474|22.587784977960474|22.587784977960474|
|2017-09-09 14:00:00|   shade|19

**SQL: Using LAST_VALUE(), FIRST_VALUE() with Window Function:**

In [97]:
spark.sql(strip_margin(
        """SELECT time, location, temp, 
          |    LAST_VALUE(temp, TRUE) OVER (ORDER BY time ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS temp_fwd_filled,
          |    FIRST_VALUE(temp, TRUE) OVER (ORDER BY time ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING) AS temp_bckd_filled
          |FROM weather
          |ORDER BY time
        """)).show()

+-------------------+--------+------------------+------------------+------------------+
|               time|location|              temp|   temp_fwd_filled|  temp_bckd_filled|
+-------------------+--------+------------------+------------------+------------------+
|2017-09-09 12:00:00|   shade|18.830184076113213|18.830184076113213|18.830184076113213|
|2017-09-09 12:01:00|     sun|              null|18.830184076113213| 21.55237663805009|
|2017-09-09 12:30:00|   shade|              null|18.830184076113213| 21.55237663805009|
|2017-09-09 12:31:00|     sun| 21.55237663805009| 21.55237663805009| 21.55237663805009|
|2017-09-09 13:00:00|   shade| 18.59059750682235| 18.59059750682235| 18.59059750682235|
|2017-09-09 13:01:00|     sun|              null| 18.59059750682235|22.587784977960474|
|2017-09-09 13:30:00|   shade|              null| 18.59059750682235|22.587784977960474|
|2017-09-09 13:31:00|     sun|22.587784977960474|22.587784977960474|22.587784977960474|
|2017-09-09 14:00:00|   shade|19

[Back to Top](#Table-of-contents)

### 5.3 Forward-fill and Backward-fill missing data with a threshold:

Pandas Dataframe also provides the flexibility to provide a threshold for forward and backward fill. Supppose, we do not want to forward fill more than two consecutive rows with null values. In that case we can mention the `threshold` paramater as follows:
`
df_filled = df.groupby('location').apply(lambda group: group.fillna(method='ffill', threshold=2))
`

We can very well achieve this using using an integer value with the `preceding` attribute of `window`. For forward fill in Spark we define a window, which is ordered in time, and which includes all the rows from the bounded by a upper bound up until the current row. We achieve this here simply by selecting the rows in the window as being the `rowsBetween -2` (at most the last two rows up), and `Window.currentRow` (the current row). Similarly for backward fill we define a window, which is ordered in time, and which includes all the rows from the current row to up until to a point bounded by a lower bound. We achieve this here simply by selecting the rows in the window as being the `rowsBetween Window.currentRow` (the current row) and `2 Following` (at most the first two rows down).

In [98]:
weather = [    
    ('2017-09-09 12:00:00', 'shade', 18.830184076113213),
    ('2017-09-09 12:01:00', 'sun',   None),
    ('2017-09-09 12:30:00', 'shade', None),
    ('2017-09-09 12:31:00', 'sun',   21.55237663805009),
    ('2017-09-09 13:00:00', 'shade', 18.59059750682235),
    ('2017-09-09 13:01:00', 'sun',   None),
    ('2017-09-09 13:30:00', 'shade', None),
    ('2017-09-09 13:31:00', 'sun',   22.587784977960474),
    ('2017-09-09 14:00:00', 'shade', None),
    ('2017-09-09 14:01:00', 'sun',   20.548896316341516),
    ('2017-09-09 15:00:00', 'shade', None),
    ('2017-09-09 15:01:00', 'sun',   None),
    ('2017-09-09 16:30:00', 'shade', None),
    ('2017-09-09 16:31:00', 'sun',   None),
    ('2017-09-09 17:00:00', 'shade', 18.101003724324197),
    ('2017-09-09 17:01:00', 'sun',   None)
]

In [99]:
weather_df = spark.createDataFrame(weather, ['time', 'location', 'temp']).cache()

In [100]:
weather_df.show()

+-------------------+--------+------------------+
|               time|location|              temp|
+-------------------+--------+------------------+
|2017-09-09 12:00:00|   shade|18.830184076113213|
|2017-09-09 12:01:00|     sun|              null|
|2017-09-09 12:30:00|   shade|              null|
|2017-09-09 12:31:00|     sun| 21.55237663805009|
|2017-09-09 13:00:00|   shade| 18.59059750682235|
|2017-09-09 13:01:00|     sun|              null|
|2017-09-09 13:30:00|   shade|              null|
|2017-09-09 13:31:00|     sun|22.587784977960474|
|2017-09-09 14:00:00|   shade|              null|
|2017-09-09 14:01:00|     sun|20.548896316341516|
|2017-09-09 15:00:00|   shade|              null|
|2017-09-09 15:01:00|     sun|              null|
|2017-09-09 16:30:00|   shade|              null|
|2017-09-09 16:31:00|     sun|              null|
|2017-09-09 17:00:00|   shade|18.101003724324197|
|2017-09-09 17:01:00|     sun|              null|
+-------------------+--------+------------------+


In [101]:
weather_df.printSchema()

root
 |-- time: string (nullable = true)
 |-- location: string (nullable = true)
 |-- temp: double (nullable = true)



**DF API: Using last(), first() with Window Function:**

In [102]:
# define the window
fwd_window = (Window.partitionBy('location')
               .orderBy('time')
               .rowsBetween(-2, Window.currentRow))

bckwd_window = (Window.partitionBy('location')
               .orderBy('time')
               .rowsBetween(Window.currentRow, 2))

In [103]:
# define the forward-filled column
ffilled_column = F.last(weather_df['temp'], ignorenulls=True).over(fwd_window) # True: fill with last non-null
bfilled_column = F.first(weather_df['temp'], ignorenulls=True).over(bckwd_window) # True: fill with first non-null

In [104]:
# do the fill
weather_df_filled = weather_df.withColumn('temp_fwd_filled', ffilled_column).withColumn('temp_bckwd_filled', bfilled_column)

In [105]:
# show
weather_df_filled.orderBy('location', 'time').show()  

+-------------------+--------+------------------+------------------+------------------+
|               time|location|              temp|   temp_fwd_filled| temp_bckwd_filled|
+-------------------+--------+------------------+------------------+------------------+
|2017-09-09 12:00:00|   shade|18.830184076113213|18.830184076113213|18.830184076113213|
|2017-09-09 12:30:00|   shade|              null|18.830184076113213| 18.59059750682235|
|2017-09-09 13:00:00|   shade| 18.59059750682235| 18.59059750682235| 18.59059750682235|
|2017-09-09 13:30:00|   shade|              null| 18.59059750682235|              null|
|2017-09-09 14:00:00|   shade|              null| 18.59059750682235|              null|
|2017-09-09 15:00:00|   shade|              null|              null|18.101003724324197|
|2017-09-09 16:30:00|   shade|              null|              null|18.101003724324197|
|2017-09-09 17:00:00|   shade|18.101003724324197|18.101003724324197|18.101003724324197|
|2017-09-09 12:01:00|     sun|  

Note: Observe how are able to restrict the Forward Filling only to next two rows. If there are three or more consecutive empty rows they remain unfilled. The same logic applies for backward filling.

**SQL: Using LAST_VALUE(), FIRST_VALUE() with Window Function:**

In [106]:
spark.sql(strip_margin(
        """SELECT time, location, temp, 
          |    LAST_VALUE(temp, TRUE) OVER (PARTITION BY location ORDER BY time ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS temp_fwd_filled,
          |    FIRST_VALUE(temp, TRUE) OVER (PARTITION BY location ORDER BY time ROWS BETWEEN CURRENT ROW AND 2 FOLLOWING) AS temp_bckd_filled
          |FROM weather
          |ORDER BY location, time
        """)).show()

+-------------------+--------+------------------+------------------+------------------+
|               time|location|              temp|   temp_fwd_filled|  temp_bckd_filled|
+-------------------+--------+------------------+------------------+------------------+
|2017-09-09 12:00:00|   shade|18.830184076113213|18.830184076113213|18.830184076113213|
|2017-09-09 12:30:00|   shade|              null|18.830184076113213| 18.59059750682235|
|2017-09-09 13:00:00|   shade| 18.59059750682235| 18.59059750682235| 18.59059750682235|
|2017-09-09 13:30:00|   shade|              null| 18.59059750682235|19.101003724324197|
|2017-09-09 14:00:00|   shade|19.101003724324197|19.101003724324197|19.101003724324197|
|2017-09-09 12:01:00|     sun|              null|              null| 21.55237663805009|
|2017-09-09 12:31:00|     sun| 21.55237663805009| 21.55237663805009| 21.55237663805009|
|2017-09-09 13:01:00|     sun|              null| 21.55237663805009|22.587784977960474|
|2017-09-09 13:31:00|     sun|22

[Back to Top](#Table-of-contents)

In [107]:
spark.stop()